# Exploratory Data Analysis and Feature Importance

In this notebook, we will:
- evaluate the datatset conditions
- look for relationships in within our data
- apply techniques to discover with features are the most important oens

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns

sns.set_style('whitegrid')

# base folders
CLEAN_DATA_DIR = Path('../clean_data/')

In [11]:
df = pd.read_csv(CLEAN_DATA_DIR / 'cleaned_data.csv', index_col=0)
df.sample(3)

,Housing Units,Median Gross Rent ($),Median Owner Cost ($),Median Home Value - Census ($),New Policies,Renewed Policies,Nonrenewed Policies (by Owner),Nonrenewed Policies (by Company),Nonrenewed Policies,Expiring Policies,...,All Disasters 5y,All Disasters 10y,Fire Disasters 1y,Fire Disasters 3y,Fire Disasters 5y,Fire Disasters 10y,Avg % White-only Pop,% Change White-only Pop,Avg Median Income,% Change Median Income
ZIP Code,,,,,,,,,,,,,,,,,,,,,
95220,2938.0,1187.0,694.0,642500.0,882.0,6216.0,634.0,203.0,837.0,7053.0,...,3.0,4.0,0.0,2.0,2.0,3.0,49.414286,-18.8,91238.428571,0.771018
94123,14908.0,2868.0,1438.0,2000001.0,1211.0,11637.0,1017.0,223.0,1240.0,12877.0,...,2.0,3.0,0.0,2.0,2.0,3.0,44.428571,-6.7,179010.714286,0.651885
90065,17135.0,1588.0,642.0,833400.0,3799.0,34204.0,2813.0,780.0,3593.0,37797.0,...,2.0,3.0,0.0,2.0,2.0,3.0,45.857143,-16.4,75247.000000,0.590669


## Pearson's Correlation

In [21]:
targets = ['% FAIR Plan Units', 'Total Exposure ($)']
features = df.columns.to_list() 

for target in targets:
    features.remove(target)

In [22]:
from scipy.stats import pearsonr

corrs = []

for col  in features:
    corr_coeff, p_value = pearsonr(df[targets[0]], df[col])
    corr_coeff2, p_value2 = pearsonr(df[targets[1]], df[col])    
    corr_coeff3, p_value3 = pearsonr(np.log(df[targets[1]]), df[col])
    corrs.append([col, corr_coeff, p_value, corr_coeff2, p_value2, corr_coeff3, p_value3])

In [23]:
results = pd.DataFrame(corrs, columns=['Feature', 'Corr FAIR Plan', 'p-value FAIR Plan',
                            'Corr Exposure', 'p-value Exposure',
                            'Corr Log-Exposure', 'p-value Log-Exposure']).sort_values(by='Corr FAIR Plan')

In [24]:
results

,Feature,Corr FAIR Plan,p-value FAIR Plan,Corr Exposure,p-value Exposure,Corr Log-Exposure,p-value Log-Exposure
28,% Market Units,-1.000000,0.000000e+00,-0.620753,4.668269e-125,-0.530221,1.923907e-85
25,Voluntary Market Units,-0.329182,7.648882e-31,-0.045870,1.176355e-01,0.053652,6.715896e-02
5,Renewed Policies,-0.303307,3.249402e-26,-0.003147,9.145529e-01,0.076909,8.635794e-03
9,Expiring Policies,-0.290855,3.794717e-24,0.013854,6.366460e-01,0.087360,2.842455e-03
0,Housing Units,-0.285613,2.625183e-23,-0.017873,5.422358e-01,0.058498,4.590791e-02
14,% Change - Renewed Policies,-0.264831,3.778062e-20,-0.167165,9.467118e-09,-0.232943,8.055819e-16
27,Total Res Units,-0.259666,2.090941e-19,0.042727,1.449852e-01,0.111221,1.424041e-04
6,Nonrenewed Policies (by Owner),-0.259152,2.474076e-19,0.028918,3.240377e-01,0.068227,1.986239e-02
1,Median Gross Rent ($),-0.205370,1.465006e-12,0.010792,7.128908e-01,0.039197,1.812369e-01
4,New Policies,-0.201175,4.204960e-12,0.104463,3.549676e-04,0.130237,8.214907e-06


In [25]:
results.to_excel(CLEAN_DATA_DIR / 'correlations.xlsx', index=False)

## Feature Importance

In [26]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(
    df,
    test_size=.2,
    random_state=13348,
    shuffle=True
)

In [27]:
from sklearn.ensemble import RandomForestRegressor

# Train two random forests to predict the FAIR Plan Units and Total Exposure
forest1 = RandomForestRegressor(n_estimators=500, max_depth=5)
forest1.fit(df_train[features], df_train[targets[0]])

forest2 = RandomForestRegressor(n_estimators=500, max_depth=5)
forest2.fit(df_train[features], df_train[targets[0]])

RandomForestRegressor(max_depth=5, n_estimators=500)

In [30]:
importance = pd.DataFrame({
    'Feature': features,
    'Imp. (% FAIR)': forest1.feature_importances_,
    'Imp. (Exposure)': forest2.feature_importances_
}).sort_values(by='Imp. (% FAIR)', ascending=False)

importance

,Feature,Imp. (% FAIR),Imp. (Exposure)
28,% Market Units,0.993340,0.993873
16,% Change - Nonrenewed Policies (by Company),0.001436,0.001333
17,% Change - Nonrenewed Policies,0.000780,0.000696
18,% Change - Expiring Policies,0.000721,0.000451
15,% Change - Nonrenewed Policies (by Owner),0.000498,0.000401
45,Avg Median Income,0.000273,0.000135
23,% Change - Claims (Fire and Smoke),0.000201,0.000122
46,% Change Median Income,0.000188,0.000285
12,% Nonrenewed Policies (by Company),0.000143,0.000163
30,Zillow Home Value 2019 ($),0.000121,0.000082


In [31]:
importance.to_csv (CLEAN_DATA_DIR / 'feature_importance.csv', index=False)